In [2]:
#Entropy of a partition
from typing import List
import math

def entropy(class_probabilities: List[float]) -> float:
    return sum(-p * math.log(p, 2)
               for p in class_probabilities 
               if p > 0)

from typing import Any
from collections import Counter

def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]
    
def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

def partition_entropy(subsets:List[List[Any]]) -> float:
    total_count = sum(len(subset) for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count
               for subset in subsets)
    
    

In [3]:
    
# Create a Decision Tree
from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level:str
    lang:str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None 

#                 level     lang    tweets  phd   did well
inputs=[Candidate('Senior', 'Java', False, False, False),
        Candidate('Senior', 'Java', False, True, False),
        Candidate('Mid', 'Python', False, False, True),
        Candidate('Junior', 'Python', False, False, True),
        Candidate('Junior', 'R', True, False, True),
        Candidate('Junior', 'R', True, True, False),
        Candidate('Mid', 'R', True, True, True),
        Candidate('Senior', 'Python', False, False, False),
        Candidate('Senior', 'R', True, False, True),
        Candidate('Junior', 'Python', True, False, True),
        Candidate('Senior', 'Python', True, True, True),
        Candidate('Mid', 'Python', False, True, True),
        Candidate('Mid', 'Java', True, False, True),
        Candidate('Junior', 'Python', False, True, False)
        ]


In [4]:

from typing import Dict, TypeVar
from collections import defaultdict

T= TypeVar('T')

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """Partition the inputs into lists"""
    partitions:Dict[Any, List[T]]=defaultdict(list)
    for input in inputs:
        key=getattr(input, attribute)#Get the value
        partitions[key].append(input)
    return partitions

def partition_entropy_by(inputs: List[Any],
                         attribute: str,
                         label_attribute: str)->float:
    """Compute the corresponding entropy"""
    partitions=partition_by(inputs, attribute)
    
    labels=[[getattr(input, label_attribute) for input in partition]
            for partition in partitions.values()]
    return partition_entropy(labels)

for key in ['level','lang','tweets','phd']:
    print(key,partition_entropy_by(inputs, key, 'did_well'))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [5]:
senior_inputs=[input for input in inputs if input.level=='Senior']
for key in ['lang','tweets','phd']:
    print(key, partition_entropy_by(senior_inputs, key, 'did_well'))

lang 0.4
tweets 0.0
phd 0.9509775004326938


In [6]:
#Put it all together
from typing import NamedTuple,Union, Any
class Leaf(NamedTuple):
    value:Any

class Split(NamedTuple):
    attribute:str
    subtrees:dict
    default_value:Any=None

DecisionTree=Union[Leaf, Split]

In [7]:
def classify(tree:DecisionTree, input:Any) -> Any: #input example: Candidate('Senior', 'Java', False, False, False)
    if isinstance(tree, Leaf):
        return tree.value
    subtree_key=getattr(input, tree.attribute)# get the value of the attribute
    if subtree_key not in tree.subtrees:
        return tree.default_value
    subtree=tree.subtrees[subtree_key]
    return classify(subtree, input) #recursion

def build_tree_3d(inputs:List[Any],
                  split_attributes:List[str],
                  target_attribute:str)->DecisionTree:
    """
    example :build_tree_3d([Candidate('Senior', 'Java', False, False, False),Candidate('Senior', 'Java', False, True, False)],
                           ['level','lang','tweets','phd'],
                           'did_well')
    """
    
    #Count target labels
    label_counts=Counter(getattr(input, target_attribute)
                         for input in inputs)#input here is a Candidate object
    """
    Counter example: Counter({'Senior': 5, 'Mid': 4, 'Junior': 5})
    """
    most_common_label=label_counts.most_common(1)[0][0]#Counter.most_common(n) example: [('Senior', 5)]
    
    #If there is only one label, return a leaf
    if len(label_counts)==1:#example: {'Senior': 5}
        return Leaf(most_common_label)
    
    #If no split attributes left, return a leaf
    if not split_attributes:# example: []
        return Leaf(most_common_label)
    
    #Otherwise split by the best attribute
    def split_entropy(attribute:str)->float:
        return partition_entropy_by(inputs, attribute, target_attribute)
    
    best_attribute=min(split_attributes, key=split_entropy)
    partitions=partition_by(inputs, best_attribute)
    """
    Partition example: {'Senior': [Candidate('Senior', 'Java', False, False, False),
                          Candidate('Senior', 'Java', False, True, False)],
                          'Mid': [Candidate('Mid', 'Python', False, False, True),
                          Candidate('Mid', 'R', True, True, True)}
    """
    new_attributes=[a for a in split_attributes if a!=best_attribute]
    subtrees={attribute_value:build_tree_3d(subset,
                                           new_attributes,
                                           target_attribute)
             for attribute_value, subset in partitions.items()}

    return Split(best_attribute, subtrees, default_value=most_common_label)


In [8]:
tree=build_tree_3d(inputs,
                     ['level','lang','tweets','phd'],
                     'did_well')
print(classify(tree, Candidate('Junior', 'Java', True, False)))
print(classify(tree, Candidate('Junior', 'Java', True, True)))
tree

True
False


Split(attribute='level', subtrees={'Senior': Split(attribute='tweets', subtrees={False: Leaf(value=False), True: Leaf(value=True)}, default_value=False), 'Mid': Leaf(value=True), 'Junior': Split(attribute='phd', subtrees={False: Leaf(value=True), True: Leaf(value=False)}, default_value=True)}, default_value=True)

In [9]:
classify(tree,Candidate("Intern","Java",True,True))

True